In [4]:
import argparse
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture

import torch
import numpy as np 

from preprocess import get_mnist, get_webcam
from train import TrainerVaDE

In [5]:
class Args:
    batch_size = 128
    lr = 1e-5
    dataset = 'webcam'
    pretrained_path = 'weights/pretrained_parameter.pth'
    patience = 50
    pretrain = True
    epochs = 200
    n_shots = 1


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    
args = Args() # Parsing all the arguments for the training
if args.dataset == 'mnist':
    dataloader_sup,  dataloader_unsup = get_mnist(args)
    n_classes = 10
else:
    dataloader_sup,  dataloader_unsup = get_webcam(args)
    n_classes = 31

In [11]:
if args.dataset == 'webcam':
    from models_office import Autoencoder, VaDE, feature_extractor
    autoencoder = Autoencoder().to(device)
    checkpoint = torch.load('autoencoder_parameters.pth.tar',
                            map_location=device)
    autoencoder.load_state_dict(checkpoint['state_dict'], strict=False)
    checkpoint = torch.load('feature_extractor_params.pth.tar',
                             map_location=device)
    feature_extractor = feature_extractor().to(device)
    feature_extractor.load_state_dict(checkpoint['state_dict'])

In [18]:
x = torch.cat([data[0] for data in dataloader_unsup]).to(device) #all x samples.
if args.dataset == 'webcam':
    x = feature_extractor(x)
    x = x.detach()
z = autoencoder.encode(x)
gmm = GaussianMixture(n_components=n_classes, covariance_type='diag')
gmm.fit(z.cpu().detach().numpy())

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                means_init=None, n_components=31, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [19]:
x_sup = torch.cat([data[0] for data in dataloader_sup]).to(device)
y_sup = torch.cat([data[1] for data in dataloader_sup]).to(device)

In [29]:
x_sup = x_sup[np.argsort(y_sup)]
y_sup = y_sup[np.argsort(y_sup)]

In [30]:
if args.dataset == 'webcam':
    x = feature_extractor(x_sup)
    x = x.detach()
z_sup = autoencoder.encode(x)
probas = gmm.predict_proba(z_sup.cpu().detach().numpy())

In [53]:


print(nums)
print(assignation)

[]
[11, 16, 2, 22, 5, 29, 21, 10, 9, 26, 4, 24, 23, 27, 6, 12, 3, 1, 8, 20, 15, 25, 30, 13, 19, 14, 28, 0, 18, 17, 7]


In [56]:
i = 0
j = 1
assignation = []
nums = np.arange(31)
for proba in probas:
    sorted_ = np.argsort(proba)
    max_ = sorted_[-1]
    if max_ in nums:
        assignation.append(max_)
        nums = np.setdiff1d(nums, max_)
        print(nums)
        print(assignation)
    else:
        print('sorted ', sorted_)


print(nums)
print(assignation)

[ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
[11]
[ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20 21 22 23 24 25
 26 27 28 29 30]
[11, 16]
[ 0  1  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20 21 22 23 24 25 26
 27 28 29 30]
[11, 16, 2]
[ 0  1  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20 21 23 24 25 26 27
 28 29 30]
[11, 16, 2, 22]
sorted  [ 7  0 11  9 14 16 20 18 23  4 28 25 30 17 10 24  3  1 15 12  8 13 26 19
 21 29 22  6 27  5  2]
[ 0  1  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20 21 23 24 25 26 27
 28 30]
[11, 16, 2, 22, 29]
[ 0  1  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20 23 24 25 26 27 28
 30]
[11, 16, 2, 22, 29, 21]
[ 0  1  3  4  5  6  7  8  9 12 13 14 15 17 18 19 20 23 24 25 26 27 28 30]
[11, 16, 2, 22, 29, 21, 10]
[ 0  1  3  4  5  6  7  8 12 13 14 15 17 18 19 20 23 24 25 26 27 28 30]
[11, 16, 2, 22, 29, 21, 10, 9]
sorted  [ 7  0 17 27  9 28 11 10 30  4 18 15 25  5 13 19  8  3 24 14 21 23 12  6
 29 22 20  2